In [103]:
#importing the libaries for parsing the files and writing capabilities
import pandas as pd
import csv
import cx_Oracle

TempDF = pd.DataFrame(columns=['ID','Name',"Serving_size-(g)","Calories-(kcal)",
                               "Water-(g/ml)","Carbs-(g)","Sugar-(g)","Fiber-(g)",
                               "Protein-(g)", "Of Which NonEssential-(g)",
                               "Fat-(g)","Of which UnHealthy-(g)","Cholesterol-(mg)"])
#opening the file for cleaning and modifying purposes
file = csv.reader(open('FYPDataset.csv'), delimiter=',')
for line in file:
    #creating the list of the fat products
    FatList = []
    FatList.append(float(line[37].replace("g","").strip()))#saturated_fatty_acids
    FatList.append(float(line[40].replace("mg","").strip())/1000)#fatty_acids_total_trans
    
    UnhealthyFat = 0.0
    for item in FatList:
        UnhealthyFat = UnhealthyFat + item
    #carbs list
    CarbsList = []
    CarbsList.append(line[27])
    sugar = line[29]
    #protein
    NonEssProtein = []
    NonEssProtein.append(line[8])
    NonEssProtein.append(line[9])
    NonEssProtein.append(line[10])
    NonEssProtein.append(line[11])
    NonEssProtein.append(line[12])
    NonEssProtein.append(line[13])
    NonEssProtein.append(line[15])
    NonEssProtein.append(line[21])
    NonEssProtein.append(line[22])
    NonEssProtein.append(line[25])
    
    NonEssentialProtein = 0.0
    for item in NonEssProtein:
        NonEssentialProtein = NonEssentialProtein + float(item.replace("g","").strip())

    AssignedProtein = round(NonEssentialProtein,3)
    
    #the name of the product
    LineToWorkWith = line[1]
    Word = ""
    LineArray = LineToWorkWith.split(",")
    for word in reversed(LineArray):
        Word = Word + word + " "
    TempDF = TempDF.append({'ID': line[0], 'Name': Word.replace("  "," ").strip(), 
                            "Serving_size-(g)": int(line[2].replace("g","").strip()), 
                            "Calories-(kcal)": int(line[3]), "Water-(g/ml)":float(line[41].replace("g","").strip()),
                            "Carbs-(g)":float(line[27].replace("g","").strip()),
                            "Sugar-(g)":float(line[29].replace("g","").strip()),
                            "Fiber-(g)":float(line[28].replace("g","").strip()),
                            "Protein-(g)":float(line[7].replace("g","").strip()), 
                            "Of Which NonEssential-(g)": AssignedProtein,
                            "Fat-(g)":float(line[36].replace("g","").strip()), 
                            "Of which UnHealthy-(g)":UnhealthyFat,
                            "Cholesterol-(mg)":float(line[6].replace("mg","").strip()),
                           }, ignore_index=True)

TempDF.to_csv("FinalCsv.csv")

In [104]:
#Connecting to localhost for testing purposes
con = cx_Oracle.connect('StagingFYPData/StagingFYPData@localhost')
cur = con.cursor()

with open("FinalCsv.csv", "r") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for lines in csv_reader:
        cur.execute(
            "insert into FoodStage (FoodStageID, FoodStageSK, FoodStageNAME, ServingAmountInGrams, Calories, WaterInMLAndGrams,CarbsINGrams,SugarInGrams,FiberInGrams,ProteinInGrams,NonEssProteinInGrams, FatInGrams, SaturatedFatInGrams, CholesterolInMicrograms) values (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14)",
            (lines[0], lines[1], lines[2], lines[3], lines[4], lines[5],lines[6], lines[7], lines[8],lines[9], lines[10], lines[11],lines[12], lines[13]))

cur.close()
con.commit()
con.close()